In [1]:
import gymnasium as gym
import ale_py

Setting up variables

In [4]:
ENV_NAME = 'ALE/Bowling-ram-v5'

Testing environnement

In [6]:
env = gym.make(ENV_NAME)

state, _ = env.reset()
print(f"Initial state: \n{state}")
print(f"Observation space: \n{env.observation_space}")
print(f"Action space: {env.action_space}")

a = env.action_space.sample()
event = env.step(a)
print('Output from applying action {} on environment:\nstate:'.format(a) \
      + '{}\nreward: {}\ndone: {}\ntruncated: {}\ninfo: {}'.format(*event))

Initial state: 
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[180 122  48]
  [180 122  48]
  [180 122  48]
  ...
  [180 122  48]
  [180 122  48]
  [180 122  48]]

 [[180 122  48]
  [180 122  48]
  [180 122  48]
  ...
  [180 122  48]
  [180 122  48]
  [180 122  48]]

 [[180 122  48]
  [180 122  48]
  [180 122  48]
  ...
  [180 122  48]
  [180 122  48]
  [180 122  48]]]
Observation space: 
Box(0, 255, (210, 160, 3), uint8)
Action space: Discrete(6)
Output from applying action 3 on environment:
state:[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0